In [0]:
from google.colab import drive
drive.mount("/content/gdrive")
import os
list=os.listdir("gdrive")
os.chdir("gdrive/"+list[0]+"/AI_Course")

In [0]:
os.chdir("Digits_pytorch/")
!ls


In [0]:
import numpy as np
import math
import os
import random
import torch.utils.data as data_utils
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import h5py
import torch
import torch.optim as optim


In [0]:
class RawDataset(data_utils.Dataset):
    def __init__(self,raw_file,batch_size):
        self.raw_file = raw_file 
        self.batch_size=batch_size
        self.h5=h5py.File(self.raw_file,'r')
        self.keys=self.h5.keys()
    def __len__(self):
        """Denotes the total number of utterances
        """
        return int(len(self.keys)/self.batch_size)

    def __getitem__(self, index):
        samples=random.sample(self.keys,self.batch_size)
        data=[]
        for i in range(0,len(samples)):
         data.append(self.h5[samples[i]][:])
        return data

class ManytooneLstm(nn.Module):
 def __init__(self,hidden_cell_dim,inp_dim,out_dim,c0,h0):
  super(ManytooneLstm,self).__init__()
  self.hidden_cell_dim=hidden_cell_dim
  self.inp_dim=inp_dim
  self.out_dim=out_dim
  self.lstm=nn.LSTM(self.inp_dim,self.hidden_cell_dim,num_layers=2,batch_first=True)
  self.linear=nn.Linear(self.hidden_cell_dim,self.out_dim)
  self.log_softmax=nn.LogSoftmax(dim=1)
  self.c0=c0
  self.h0=h0
 def forward(self,batch_data):
  rnn_out,_=self.lstm(batch_data,(self.h0,self.c0))
  out=rnn_out[:,-1,:].view(batch_data.shape[0],-1)
  scores=self.log_softmax(self.linear(out))
  return scores

In [0]:
use_cuda = not False and torch.cuda.is_available()
print(use_cuda)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

True
cuda


In [0]:
h0 = torch.randn(2,64,128).cuda(device)
c0= torch.randn(2,64,128).cuda(device)
model =ManytooneLstm(128,39,10,c0,h0).cuda(device)
loss_fun=nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
data_set_train=RawDataset("feats_labels_new_train.h5",64)
data_set_dev=RawDataset("feats_labels_new_dev.h5",64)
data_set_test=RawDataset("feats_labels_new_test.h5",64)
train_loader = data_utils.DataLoader(data_set_train,batch_size=1,shuffle=True)
dev_loader = data_utils.DataLoader(data_set_dev,batch_size=1,shuffle=False)
test_loader = data_utils.DataLoader(data_set_test,batch_size=1,shuffle=False)

In [0]:
count=0
for epoch in range(0,30):
 tot_loss=0
 dev_loss=0
 for d in train_loader:
  data_batch=[]
  labels_batch=[]
  for label in range(0,len(d)):
   data_batch.append(d[label][:,:,:-1].view(-1,39))
   labels_batch.append(d[label][:,0,-1][0])
 #print(data_batch[0])
  batch_data_tensor=torch.tensor(pad_sequence(data_batch).transpose(0,1).clone().detach(),dtype=torch.float32)
  batch_labels_tensor=torch.tensor(labels_batch,dtype=torch.long)
  predictions=model(batch_data_tensor.cuda(device))
  loss=loss_fun(predictions,batch_labels_tensor.cuda(device))
  tot_loss=tot_loss+loss.item()
  optimizer.zero_grad()
  loss.backward()testing_online
  optimizer.step()
  count=count+1
 for d in dev_loader:
  data_batch=[]
  labels_batch=[]
  for label in range(0,len(d)):
   data_batch.append(d[label][:,:,:-1].view(-1,39))
   labels_batch.append(d[label][:,0,-1][0])
  batch_data_tensor=torch.tensor(pad_sequence(data_batch).transpose(0,1).clone().detach(),dtype=torch.float32)
  batch_labels_tensor=torch.tensor(labels_batch,dtype=torch.long)
  predictions=model(batch_data_tensor.cuda(device))
  loss=loss_fun(predictions,batch_labels_tensor.cuda(device))
  dev_loss=dev_loss+loss.item()
 print("epoch:",epoch,"train_loss:",tot_loss,"dev_loss",dev_loss)
 if epoch % 10 == 0:
  torch.save(model.state_dict(),"model"+str(epoch)+".pt")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


epoch: 0 train_loss: 622.1106490492821 dev_loss 65.9388701915741
epoch: 1 train_loss: 224.92398220300674 dev_loss 30.132918141782284
epoch: 2 train_loss: 121.02582162246108 dev_loss 22.264910973608494
epoch: 3 train_loss: 81.46238598227501 dev_loss 16.510986488312483
epoch: 4 train_loss: 62.282275553792715 dev_loss 17.239564817398787
epoch: 5 train_loss: 50.59651064872742 dev_loss 13.213451493531466
epoch: 6 train_loss: 40.2070733550936 dev_loss 14.329188887029886
epoch: 7 train_loss: 34.87017427012324 dev_loss 10.945388250052929
epoch: 8 train_loss: 29.959842586889863 dev_loss 11.909979276359081
epoch: 9 train_loss: 28.75616285018623 dev_loss 11.9822404012084
epoch: 10 train_loss: 24.579663449898362 dev_loss 11.936259508132935
epoch: 11 train_loss: 23.586732091382146 dev_loss 11.727192714810371
epoch: 12 train_loss: 19.508164515718818 dev_loss 12.340268567204475
epoch: 13 train_loss: 22.432873932644725 dev_loss 14.096912488341331
epoch: 14 train_loss: 17.290733851492405 dev_loss 11.78